In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def preprocess_data(file_path, crypto_symbol, features):
    """
    Preprocess the cryptocurrency data for LSTM input.
    """
    # Load and filter data
    crypto_data = pd.read_csv(file_path)
    crypto_data['Date'] = pd.to_datetime(crypto_data['Date'])
    crypto_data.set_index('Date', inplace=True)
    crypto_series = crypto_data[crypto_data['Symbol'] == crypto_symbol][features]

    # Feature engineering
    crypto_series['Moving_Avg_7'] = crypto_series['Adj'].rolling(window=7).mean()
    crypto_series['Moving_Avg_14'] = crypto_series['Adj'].rolling(window=14).mean()
    crypto_series['Moving_Avg_30'] = crypto_series['Adj'].rolling(window=30).mean()
    
    crypto_series.fillna(method='bfill', inplace=True)

    # Normalize data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(crypto_series)

    return scaled_data, scaler, crypto_series


def run_lstm_forecast(scaled_data, scaler, crypto_series, sequence_length=30):
    """
    Build, train, and evaluate the LSTM model, including high/low forecasts.
    """
    # Create sequences
    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, :-1])  # All features except the target (Adj)
        y.append(scaled_data[i, 0])  # The target (Adj)
    X, y = np.array(X), np.array(y)

    # Train-test split
    train_size = int(0.8 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, verbose=1)

    # Predictions
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(
        np.hstack((predictions, np.zeros((predictions.shape[0], X_test.shape[2]))))
    )[:, 0]

    # Inverse transform actual test data
    actual = scaler.inverse_transform(
        np.hstack((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], X_test.shape[2]))))
    )[:, 0]

    # Predict highs and lows
    high_forecast = np.max(predictions)  
    low_forecast = np.min(predictions) 

    # Prepare highs and lows for table
    future_dates = pd.date_range(start=crypto_series.index[-1], periods=len(predictions) + 1, freq='D')[1:]
    high_low_df = pd.DataFrame({
        "Date": future_dates,
        "High": [high_forecast] * len(future_dates),
        "Low": [low_forecast] * len(future_dates),
    })

    # Calculate MAPE and accuracy
    mape = np.mean(np.abs((actual - predictions) / actual)) * 100
    accuracy = 100 - mape  # Accuracy in percentage
    print(f"Model Accuracy: {accuracy:.2f}%")

    # Add buy and sell signals with Moving_Avg
    buy_signals = crypto_series[(crypto_series['Moving_Avg_7'] > crypto_series['Moving_Avg_30']) &
                                (crypto_series['Moving_Avg_7'] > crypto_series['Moving_Avg_14']) &
                                (crypto_series['Moving_Avg_7'].shift(1) <= crypto_series['Moving_Avg_30'].shift(1))]
    sell_signals = crypto_series[(crypto_series['Moving_Avg_7'] < crypto_series['Moving_Avg_30']) &
                                 (crypto_series['Moving_Avg_7'] < crypto_series['Moving_Avg_14']) &
                                 (crypto_series['Moving_Avg_7'].shift(1) >= crypto_series['Moving_Avg_30'].shift(1))]

    # Plot
    fig = go.Figure()

    #  candlestick chart
    fig.add_trace(go.Candlestick(
        x=crypto_series.index,
        open=crypto_series['Open'],
        high=crypto_series['High'],
        low=crypto_series['Low'],
        close=crypto_series['Adj'],
        name="OHLC"
    ))

    #  predicted data
    fig.add_trace(go.Scatter(
        x=crypto_series.index[-len(y_test):],
        y=predictions,
        mode='lines',
        name='Predicted Data',
        line=dict(color='orange', dash='dot')
    ))

    #  moving averages
    fig.add_trace(go.Scatter(
        x=crypto_series.index,
        y=crypto_series['Moving_Avg_7'],
        mode='lines',
        name='7-Day MA',
        line=dict(color='blue', width=1.5)
    ))
    fig.add_trace(go.Scatter(
        x=crypto_series.index,
        y=crypto_series['Moving_Avg_14'],
        mode='lines',
        name='14-Day MA',
        line=dict(color='purple', width=1.5)
    ))
    fig.add_trace(go.Scatter(
        x=crypto_series.index,
        y=crypto_series['Moving_Avg_30'],
        mode='lines',
        name='30-Day MA',
        line=dict(color='green', width=1.5)
    ))

    #  buy signals
    fig.add_trace(go.Scatter(
        x=buy_signals.index,
        y=buy_signals['Adj'],
        mode='markers',
        name='Buy Signals',
        marker=dict(color='green', size=10, symbol='triangle-up')
    ))

    #  sell signals
    fig.add_trace(go.Scatter(
        x=sell_signals.index,
        y=sell_signals['Adj'],
        mode='markers',
        name='Sell Signals',
        marker=dict(color='red', size=10, symbol='triangle-down')
    ))

    # Update layout
    fig.update_layout(
        title="Interactive Forecast with LSTM Model and Buy/Sell Signals",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        xaxis_rangeslider_visible=True,
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    return predictions, actual, accuracy, high_low_df, fig